In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("HiveIntegration") \
    .config("spark.sql.catalogImplementation", "hive") \
    .config("spark.sql.warehouse.dir", "hdfs://localhost:9000/user/hive/warehouse") \
    .enableHiveSupport() \
    .getOrCreate()

spark

In [9]:
#Extracted data from local into datafraem

df = spark.read.csv("file:///D:/Apache_spark/datasets/emp_dept.csv", header=True, inferSchema = True)

df.printSchema()
df.count()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- Joining_Date: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Location: string (nullable = true)



15

In [ ]:
# loaded data in landing location

df.write.format("csv").option("header", True).save("hdfs://localhost:9000/datasets/emp_data.csv")

In [6]:
df.rdd.getNumPartitions()



1

In [15]:
# created raw temp table and loaded data from landing location.

spark.sql("""

CREATE TABLE Raw_temp_table(
    ID INT,
    Name STRING,
    Age STRING,
    Email STRING,
    Salary STRING,
    Joining_Date STRING,
    Status STRING,
    Department STRING,
    Experience INT,
    LOCATION STRING
)ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION 'hdfs://localhost:9000/datasets/emp_data.csv'
""")

spark.sql("""
LOAD DATA INPATH 'hdfs://localhost:9000/datasets/emp_data.csv' INTO TABLE Raw_temp_table;
""")


DataFrame[]

In [20]:
spark.sql("describe table Raw_temp_table").show()

+------------+---------+-------+
|    col_name|data_type|comment|
+------------+---------+-------+
|          ID|      int|   NULL|
|        Name|   string|   NULL|
|         Age|   string|   NULL|
|       Email|   string|   NULL|
|      Salary|   string|   NULL|
|Joining_Date|   string|   NULL|
|      Status|   string|   NULL|
|  Department|   string|   NULL|
|  Experience|      int|   NULL|
|    LOCATION|   string|   NULL|
+------------+---------+-------+



In [19]:
df.show()

+---+-------+----+-----------------+-----------+------------+--------+-----------+----------+-------------+
| ID|   Name| Age|            Email|     Salary|Joining_Date|  Status| Department|Experience|     Location|
+---+-------+----+-----------------+-----------+------------+--------+-----------+----------+-------------+
|101|  Alice|  25|  alice@email.com|      50000|  2022-06-15|  Active|         HR|         3|     New York|
|102|    Bob|NULL|       bob@domain|      60000|  2023-07-20|    NULL|    Finance|         5|       London|
|103|  Alice|  25|  alice@email.com|      50000|  2022-06-15|  Active|         HR|         3|     New York|
|104|Charlie|  28|charlie@email.com|     -45000|  2021-05-10|Inactive|Engineering|         7|       Berlin|
|105|  Diana|NULL|  diana@email.com|       NULL|  2019-03-25|  Active|  Marketing|      NULL|        Paris|
|106|   NULL|  30|    invalid-email|      70000|  2024-09-18|  Active|      Sales|         4|        Tokyo|
|107|  Eddie|  22|  eddie@em

In [52]:
# converting raw table data into df from trasformation

df = spark.sql("SELECT * FROM Raw_temp_table")


In [23]:
df.show()

+----+-------+----+-----------------+-----------+------------+--------+-----------+----------+-------------+
|  ID|   Name| Age|            Email|     Salary|Joining_Date|  Status| Department|Experience|     LOCATION|
+----+-------+----+-----------------+-----------+------------+--------+-----------+----------+-------------+
|NULL|   Name| Age|            Email|     Salary|Joining_Date|  Status| Department|      NULL|     Location|
| 101|  Alice|  25|  alice@email.com|      50000|  2022-06-15|  Active|         HR|         3|     New York|
| 102|    Bob|    |       bob@domain|      60000|  2023-07-20|        |    Finance|         5|       London|
| 103|  Alice|  25|  alice@email.com|      50000|  2022-06-15|  Active|         HR|         3|     New York|
| 104|Charlie|  28|charlie@email.com|     -45000|  2021-05-10|Inactive|Engineering|         7|       Berlin|
| 105|  Diana|NULL|  diana@email.com|           |  2019-03-25|  Active|  Marketing|      NULL|        Paris|
| 106|   NULL|  30|

In [24]:
from pyspark.sql.functions import col

# Define bad record conditions
bad_records_df = df.filter(
    (col("Age").isNull()) |  # Check for missing Age
    (~col("Email").rlike("^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\\.[a-zA-Z0-9-.]+$")) |  # Invalid Email
    (col("Salary").isNull()) | (col("Salary") <= 0)  # Salary must be positive
)

bad_records_df.show()

+----+-------+---+-----------------+------+------------+--------+-----------+----------+--------+
|  ID|   Name|Age|            Email|Salary|Joining_Date|  Status| Department|Experience|LOCATION|
+----+-------+---+-----------------+------+------------+--------+-----------+----------+--------+
|NULL|   Name|Age|            Email|Salary|Joining_Date|  Status| Department|      NULL|Location|
| 102|    Bob|   |       bob@domain| 60000|  2023-07-20|        |    Finance|         5|  London|
| 104|Charlie| 28|charlie@email.com|-45000|  2021-05-10|Inactive|Engineering|         7|  Berlin|
| 106|   NULL| 30|    invalid-email| 70000|  2024-09-18|  Active|      Sales|         4|   Tokyo|
| 109|   Gary| 27|                 | 75000|  2020-02-29|  Active|         HR|         2|  Mumbai|
| 112|   Kate| 29|   invalid@domain| 48000|  2022-04-18|  Active|Engineering|         5|  London|
| 113|  Louis| 26|      louis@email| 70000|  2020.03.30|Inactive|         HR|         4|  Berlin|
+----+-------+---+--

In [26]:
from pyspark.sql.functions import when, col

df_p = bad_records_df .withColumn("bad_record", 
    when((col("Age").isNull()) | (~col("Email").rlike("^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\\.[a-zA-Z0-9-.]+$")) |
         (col("Salary").isNull()) | (col("Salary") <= 0), "Corrupt").otherwise(None)
)

df_p.show()

+----+-------+---+-----------------+------+------------+--------+-----------+----------+--------+----------+
|  ID|   Name|Age|            Email|Salary|Joining_Date|  Status| Department|Experience|LOCATION|bad_record|
+----+-------+---+-----------------+------+------------+--------+-----------+----------+--------+----------+
|NULL|   Name|Age|            Email|Salary|Joining_Date|  Status| Department|      NULL|Location|   Corrupt|
| 102|    Bob|   |       bob@domain| 60000|  2023-07-20|        |    Finance|         5|  London|   Corrupt|
| 104|Charlie| 28|charlie@email.com|-45000|  2021-05-10|Inactive|Engineering|         7|  Berlin|   Corrupt|
| 106|   NULL| 30|    invalid-email| 70000|  2024-09-18|  Active|      Sales|         4|   Tokyo|   Corrupt|
| 109|   Gary| 27|                 | 75000|  2020-02-29|  Active|         HR|         2|  Mumbai|   Corrupt|
| 112|   Kate| 29|   invalid@domain| 48000|  2022-04-18|  Active|Engineering|         5|  London|   Corrupt|
| 113|  Louis| 26| 

In [29]:
# Hadling Null records with default records

df_p = df_p.na.fill({
    "ID": 102,  # Default id if missing
    "Age": 25,  # Default age if missing
    "Salary": 30000,  # Assign minimum salary
    "Status": "Active", # Assign missing user are active
    "Experience": 1,
    "Joining_Date": "2021-05-10",
    "LOCATION": "India"
})

df_p.show()

+---+-------+---+-----------------+------+------------+--------+-----------+----------+--------+----------+
| ID|   Name|Age|            Email|Salary|Joining_Date|  Status| Department|Experience|LOCATION|bad_record|
+---+-------+---+-----------------+------+------------+--------+-----------+----------+--------+----------+
|102|   Name|Age|            Email|Salary|Joining_Date|  Status| Department|         1|Location|   Corrupt|
|102|    Bob|   |       bob@domain| 60000|  2023-07-20|        |    Finance|         5|  London|   Corrupt|
|104|Charlie| 28|charlie@email.com|-45000|  2021-05-10|Inactive|Engineering|         7|  Berlin|   Corrupt|
|106|   NULL| 30|    invalid-email| 70000|  2024-09-18|  Active|      Sales|         4|   Tokyo|   Corrupt|
|109|   Gary| 27|                 | 75000|  2020-02-29|  Active|         HR|         2|  Mumbai|   Corrupt|
|112|   Kate| 29|   invalid@domain| 48000|  2022-04-18|  Active|Engineering|         5|  London|   Corrupt|
|113|  Louis| 26|      louis

In [31]:
# Hadling the bad records into good records

from pyspark.sql.functions import when

df_p = df_p.withColumn("Email", 
    when(~col("Email").rlike("^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\\.[a-zA-Z0-9-.]+$"), "unknown@email.com")
    .otherwise(col("Email"))
)

df_p = df_p.withColumn("Salary", when(col("Salary") < 0, 30000).otherwise(col("Salary")))

df_p = df_p.withColumn("Experience", when(col("Experience") < 0, 1).otherwise(col("Experience")))

df_p.show()

+---+-------+---+-----------------+------+------------+--------+-----------+----------+--------+----------+
| ID|   Name|Age|            Email|Salary|Joining_Date|  Status| Department|Experience|LOCATION|bad_record|
+---+-------+---+-----------------+------+------------+--------+-----------+----------+--------+----------+
|102|   Name|Age|unknown@email.com|Salary|Joining_Date|  Status| Department|         1|Location|   Corrupt|
|102|    Bob|   |unknown@email.com| 60000|  2023-07-20|        |    Finance|         5|  London|   Corrupt|
|104|Charlie| 28|charlie@email.com| 30000|  2021-05-10|Inactive|Engineering|         7|  Berlin|   Corrupt|
|106|   NULL| 30|unknown@email.com| 70000|  2024-09-18|  Active|      Sales|         4|   Tokyo|   Corrupt|
|109|   Gary| 27|unknown@email.com| 75000|  2020-02-29|  Active|         HR|         2|  Mumbai|   Corrupt|
|112|   Kate| 29|unknown@email.com| 48000|  2022-04-18|  Active|Engineering|         5|  London|   Corrupt|
|113|  Louis| 26|unknown@ema

In [37]:
from pyspark.sql.functions import when, col, trim

df_p = df_p.withColumn("Name", when(col("Name").isNull() | (trim(col("Name")) == ""), "Unknown").otherwise(col("Name")))
df_p = df_p.withColumn("Age", when(col("Age").isNull() | (trim(col("Age")) == ""), 25).otherwise(col("Age")))
df_p = df_p.withColumn("Salary", when(col("Salary").isNull() | (trim(col("Name")) == ""), 30000).otherwise(col("Salary")))
df_p = df_p.withColumn("Status", when(col("Status").isNull() | (trim(col("Status")) == ""), "Active").otherwise(col("Status")))
df_p = df_p.withColumn("Department", when(col("Department").isNull() | (trim(col("Department")) == ""), "Engineering").otherwise(col("Department")))

df_p.show()

+---+-------+---+-----------------+------+------------+--------+-----------+----------+--------+----------+
| ID|   Name|Age|            Email|Salary|Joining_Date|  Status| Department|Experience|LOCATION|bad_record|
+---+-------+---+-----------------+------+------------+--------+-----------+----------+--------+----------+
|102|   Name|Age|unknown@email.com|Salary|Joining_Date|  Status| Department|         1|Location|   Corrupt|
|102|    Bob| 25|unknown@email.com| 60000|  2023-07-20|  Active|    Finance|         5|  London|   Corrupt|
|104|Charlie| 28|charlie@email.com| 30000|  2021-05-10|Inactive|Engineering|         7|  Berlin|   Corrupt|
|106|   NULL| 30|unknown@email.com| 70000|  2024-09-18|  Active|      Sales|         4|   Tokyo|   Corrupt|
|109|   Gary| 27|unknown@email.com| 75000|  2020-02-29|  Active|         HR|         2|  Mumbai|   Corrupt|
|112|   Kate| 29|unknown@email.com| 48000|  2022-04-18|  Active|Engineering|         5|  London|   Corrupt|
|113|  Louis| 26|unknown@ema

In [38]:
from pyspark.sql.functions import to_date, when, col

df_p = df_p.withColumn("Joining_Date", to_date(col("Joining_Date"), "yyyy-MM-dd"))

df_p = df_p.withColumn("Salary", 
    when(col("Salary").isNull() | (~col("Salary").rlike("^[0-9]+$")) | (col("Salary") <= 0), 30000)
    .otherwise(col("Salary").cast("int"))
)

df_p.show()

+---+-------+---+-----------------+------+------------+--------+-----------+----------+--------+----------+
| ID|   Name|Age|            Email|Salary|Joining_Date|  Status| Department|Experience|LOCATION|bad_record|
+---+-------+---+-----------------+------+------------+--------+-----------+----------+--------+----------+
|102|   Name|Age|unknown@email.com| 30000|        NULL|  Status| Department|         1|Location|   Corrupt|
|102|    Bob| 25|unknown@email.com| 60000|  2023-07-20|  Active|    Finance|         5|  London|   Corrupt|
|104|Charlie| 28|charlie@email.com| 30000|  2021-05-10|Inactive|Engineering|         7|  Berlin|   Corrupt|
|106|   NULL| 30|unknown@email.com| 70000|  2024-09-18|  Active|      Sales|         4|   Tokyo|   Corrupt|
|109|   Gary| 27|unknown@email.com| 75000|  2020-02-29|  Active|         HR|         2|  Mumbai|   Corrupt|
|112|   Kate| 29|unknown@email.com| 48000|  2022-04-18|  Active|Engineering|         5|  London|   Corrupt|
|113|  Louis| 26|unknown@ema

In [40]:
df_p.na.fill({
    "Joining_Date": "2024-07-08"
})

df_p.show()

+---+-------+---+-----------------+------+------------+--------+-----------+----------+--------+----------+
| ID|   Name|Age|            Email|Salary|Joining_Date|  Status| Department|Experience|LOCATION|bad_record|
+---+-------+---+-----------------+------+------------+--------+-----------+----------+--------+----------+
|102|   Name|Age|unknown@email.com| 30000|        NULL|  Status| Department|         1|Location|   Corrupt|
|102|    Bob| 25|unknown@email.com| 60000|  2023-07-20|  Active|    Finance|         5|  London|   Corrupt|
|104|Charlie| 28|charlie@email.com| 30000|  2021-05-10|Inactive|Engineering|         7|  Berlin|   Corrupt|
|106|   NULL| 30|unknown@email.com| 70000|  2024-09-18|  Active|      Sales|         4|   Tokyo|   Corrupt|
|109|   Gary| 27|unknown@email.com| 75000|  2020-02-29|  Active|         HR|         2|  Mumbai|   Corrupt|
|112|   Kate| 29|unknown@email.com| 48000|  2022-04-18|  Active|Engineering|         5|  London|   Corrupt|
|113|  Louis| 26|unknown@ema

In [41]:
from pyspark.sql.functions import when, col

df_p = df_p.withColumn("Joining_Date", 
    when(col("Joining_Date").isNull() | (col("Joining_Date") == ""), "2024-07-08")
    .otherwise(col("Joining_Date"))
)

In [43]:
df_p.show()

+---+-------+---+-----------------+------+------------+--------+-----------+----------+--------+----------+
| ID|   Name|Age|            Email|Salary|Joining_Date|  Status| Department|Experience|LOCATION|bad_record|
+---+-------+---+-----------------+------+------------+--------+-----------+----------+--------+----------+
|102|   Name|Age|unknown@email.com| 30000|  2024-07-08|  Status| Department|         1|Location|   Corrupt|
|102|    Bob| 25|unknown@email.com| 60000|  2023-07-20|  Active|    Finance|         5|  London|   Corrupt|
|104|Charlie| 28|charlie@email.com| 30000|  2021-05-10|Inactive|Engineering|         7|  Berlin|   Corrupt|
|106|   NULL| 30|unknown@email.com| 70000|  2024-09-18|  Active|      Sales|         4|   Tokyo|   Corrupt|
|109|   Gary| 27|unknown@email.com| 75000|  2020-02-29|  Active|         HR|         2|  Mumbai|   Corrupt|
|112|   Kate| 29|unknown@email.com| 48000|  2022-04-18|  Active|Engineering|         5|  London|   Corrupt|
|113|  Louis| 26|unknown@ema

In [46]:
df_p = df_p.drop("bad_record")

In [47]:
df_p.show()

+---+-------+---+-----------------+------+------------+--------+-----------+----------+--------+
| ID|   Name|Age|            Email|Salary|Joining_Date|  Status| Department|Experience|LOCATION|
+---+-------+---+-----------------+------+------------+--------+-----------+----------+--------+
|102|   Name|Age|unknown@email.com| 30000|  2024-07-08|  Status| Department|         1|Location|
|102|    Bob| 25|unknown@email.com| 60000|  2023-07-20|  Active|    Finance|         5|  London|
|104|Charlie| 28|charlie@email.com| 30000|  2021-05-10|Inactive|Engineering|         7|  Berlin|
|106|   NULL| 30|unknown@email.com| 70000|  2024-09-18|  Active|      Sales|         4|   Tokyo|
|109|   Gary| 27|unknown@email.com| 75000|  2020-02-29|  Active|         HR|         2|  Mumbai|
|112|   Kate| 29|unknown@email.com| 48000|  2022-04-18|  Active|Engineering|         5|  London|
|113|  Louis| 26|unknown@email.com| 70000|  2024-07-08|Inactive|         HR|         4|  Berlin|
+---+-------+---+-------------

In [49]:
df_transformed = df_p.toDF("id", "name", "age", "email", "salary", "joining_date", "status", "department", "experience", "location")

df_transformed.show()

+---+-------+---+-----------------+------+------------+--------+-----------+----------+--------+
| id|   name|age|            email|salary|joining_date|  status| department|experience|location|
+---+-------+---+-----------------+------+------------+--------+-----------+----------+--------+
|102|   Name|Age|unknown@email.com| 30000|  2024-07-08|  Status| Department|         1|Location|
|102|    Bob| 25|unknown@email.com| 60000|  2023-07-20|  Active|    Finance|         5|  London|
|104|Charlie| 28|charlie@email.com| 30000|  2021-05-10|Inactive|Engineering|         7|  Berlin|
|106|   NULL| 30|unknown@email.com| 70000|  2024-09-18|  Active|      Sales|         4|   Tokyo|
|109|   Gary| 27|unknown@email.com| 75000|  2020-02-29|  Active|         HR|         2|  Mumbai|
|112|   Kate| 29|unknown@email.com| 48000|  2022-04-18|  Active|Engineering|         5|  London|
|113|  Louis| 26|unknown@email.com| 70000|  2024-07-08|Inactive|         HR|         4|  Berlin|
+---+-------+---+-------------

In [54]:
df_original_clean = df.filter(
    col("Salary").isNotNull() & (~col("Salary").rlike("^[^0-9]+$")) & (col("Salary") > 0) &
    col("Joining_Date").isNotNull()
)

df_original_clean.show()

+---+-----+----+---------------+------+------------+--------+-----------+----------+-------------+
| ID| Name| Age|          Email|Salary|Joining_Date|  Status| Department|Experience|     LOCATION|
+---+-----+----+---------------+------+------------+--------+-----------+----------+-------------+
|101|Alice|  25|alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|     New York|
|102|  Bob|    |     bob@domain| 60000|  2023-07-20|        |    Finance|         5|       London|
|103|Alice|  25|alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|     New York|
|106| NULL|  30|  invalid-email| 70000|  2024-09-18|  Active|      Sales|         4|        Tokyo|
|107|Eddie|  22|eddie@email.com| 40000|            |  Active|         IT|         1|San Francisco|
|108|Fiona|  26|fiona@email.com| 55000|  2023-11-05|    NULL|    Finance|         6|       Sydney|
|109| Gary|  27|               | 75000|  2020-02-29|  Active|         HR|         2|       Mumbai|
|110|Helen

In [56]:
from pyspark.sql.functions import col

df_good_records = df.filter(
    (col("Salary").isNotNull()) & (col("Salary").rlike("^[0-9]+$")) & (col("Salary") > 0) & 
    (col("Joining_Date").isNotNull()) & (col("Joining_Date") != "") & 
    (col("Email").rlike("^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\\.[a-zA-Z0-9-.]+$"))
)

df_good_records.show()

+---+-----+----+---------------+------+------------+--------+----------+----------+--------+
| ID| Name| Age|          Email|Salary|Joining_Date|  Status|Department|Experience|LOCATION|
+---+-----+----+---------------+------+------------+--------+----------+----------+--------+
|101|Alice|  25|alice@email.com| 50000|  2022-06-15|  Active|        HR|         3|New York|
|103|Alice|  25|alice@email.com| 50000|  2022-06-15|  Active|        HR|         3|New York|
|108|Fiona|  26|fiona@email.com| 55000|  2023-11-05|    NULL|   Finance|         6|  Sydney|
|110|Helen|NULL|helen@email.com| 45000|  2018-06-07|Inactive| Marketing|         8|   Dubai|
|111| John|    | john@email.com| 80000|  2021-11-10|  Active|        IT|         3|New York|
|115| Nina|  28| nina@email.com| 55000|    06/15/22|  Active| Marketing|         6|   Tokyo|
+---+-----+----+---------------+------+------------+--------+----------+----------+--------+



In [57]:
df1 = df_good_records.join(df_transformed, df["ID"] == df_transformed["id"], how="left").show()

+---+-----+----+---------------+------+------------+--------+----------+----------+--------+----+----+----+-----+------+------------+------+----------+----------+--------+
| ID| Name| Age|          Email|Salary|Joining_Date|  Status|Department|Experience|LOCATION|  id|name| age|email|salary|joining_date|status|department|experience|location|
+---+-----+----+---------------+------+------------+--------+----------+----------+--------+----+----+----+-----+------+------------+------+----------+----------+--------+
|101|Alice|  25|alice@email.com| 50000|  2022-06-15|  Active|        HR|         3|New York|NULL|NULL|NULL| NULL|  NULL|        NULL|  NULL|      NULL|      NULL|    NULL|
|103|Alice|  25|alice@email.com| 50000|  2022-06-15|  Active|        HR|         3|New York|NULL|NULL|NULL| NULL|  NULL|        NULL|  NULL|      NULL|      NULL|    NULL|
|108|Fiona|  26|fiona@email.com| 55000|  2023-11-05|    NULL|   Finance|         6|  Sydney|NULL|NULL|NULL| NULL|  NULL|        NULL|  NULL|

In [95]:
df_r = df_good_records.union(df_transformed)
df_r.show()


+---+-------+----+-----------------+------+------------+--------+-----------+----------+--------+
| ID|   Name| Age|            Email|Salary|Joining_Date|  Status| Department|Experience|LOCATION|
+---+-------+----+-----------------+------+------------+--------+-----------+----------+--------+
|101|  Alice|  25|  alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|New York|
|103|  Alice|  25|  alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|New York|
|108|  Fiona|  26|  fiona@email.com| 55000|  2023-11-05|    NULL|    Finance|         6|  Sydney|
|110|  Helen|NULL|  helen@email.com| 45000|  2018-06-07|Inactive|  Marketing|         8|   Dubai|
|111|   John|    |   john@email.com| 80000|  2021-11-10|  Active|         IT|         3|New York|
|115|   Nina|  28|   nina@email.com| 55000|    06/15/22|  Active|  Marketing|         6|   Tokyo|
|102|   Name| Age|unknown@email.com| 30000|  2024-07-08|  Status| Department|         1|Location|
|102|    Bob|  25|un

In [96]:
df_r.count()

13

In [97]:
from pyspark.sql.functions import when, col, to_date, trim

df_r = df_r.withColumn("Joining_Date", to_date(col("Joining_Date"), "yyyy-MM-dd"))

df_r = df_r.withColumn("Joining_Date", when(col("Joining_Date").isNull() | (col("Joining_Date") == ""), "2024-07-08").otherwise(col("Joining_Date")))

df_r = df_r.withColumn("Salary", when(col("Salary").isNull() | (~col("Salary").rlike("^[0-9]+$")) | (col("Salary") <= 0) | (trim(col("Salary")) == ""), 30000).otherwise(col("Salary").cast("int")))

df_r = df_r.withColumn("Name", when(col("Name").isNull() | (trim(col("Name")) == ""), "Unknown").otherwise(col("Name")))
df_r= df_r.withColumn("Age", when(col("Age").isNull() | (trim(col("Age")) == ""), 25).otherwise(col("Age")))

df_r = df_r.withColumn("Status", when(col("Status").isNull() | (trim(col("Status")) == ""), "Active").otherwise(col("Status")))
df_r = df_r.withColumn("Department", when(col("Department").isNull() | (trim(col("Department")) == ""), "Engineering").otherwise(col("Department")))
df_r = df_r.withColumn("Email", when(~col("Email").rlike("^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\\.[a-zA-Z0-9-.]+$"), "unknown@email.com").otherwise(col("Email")))

df_r = df_r.withColumn("Experience", when(col("Experience") < 0, 1).otherwise(col("Experience")))

df_r.show()

+---+-------+----+-----------------+------+------------+--------+-----------+----------+--------+
| ID|   Name| Age|            Email|Salary|Joining_Date|  Status| Department|Experience|LOCATION|
+---+-------+----+-----------------+------+------------+--------+-----------+----------+--------+
|101|  Alice|  25|  alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|New York|
|103|  Alice|  25|  alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|New York|
|108|  Fiona|  26|  fiona@email.com| 55000|  2023-11-05|    NULL|    Finance|         6|  Sydney|
|110|  Helen|NULL|  helen@email.com| 45000|  2018-06-07|Inactive|  Marketing|         8|   Dubai|
|111|   John|  25|   john@email.com| 80000|  2021-11-10|  Active|         IT|         3|New York|
|115|   Nina|  28|   nina@email.com| 55000|  2024-07-08|  Active|  Marketing|         6|   Tokyo|
|102|   Name| Age|unknown@email.com| 30000|  2024-07-08|  Status| Department|         1|Location|
|102|    Bob|  25|un

In [86]:
df_r = df_r.withColumn("Status", when(col("Status").isNull() | (trim(col("Status")) == ""), "Active").otherwise(col("Status"))) \
           .withColumn("Age", when(col("Age").isNull() | (trim(col("Age")) == ""), 25).otherwise(col("Age")))

df_r.show()

+---+-------+----+-----------------+------+------------+--------+-----------+----------+--------+
| ID|   Name| Age|            Email|Salary|Joining_Date|  Status| Department|Experience|LOCATION|
+---+-------+----+-----------------+------+------------+--------+-----------+----------+--------+
|101|  Alice|  25|  alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|New York|
|103|  Alice|  25|  alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|New York|
|108|  Fiona|  26|  fiona@email.com| 55000|  2023-11-05|    NULL|    Finance|         6|  Sydney|
|110|  Helen|NULL|  helen@email.com| 45000|  2018-06-07|Inactive|  Marketing|         8|   Dubai|
|111|   John|  25|   john@email.com| 80000|  2021-11-10|  Active|         IT|         3|New York|
|115|   Nina|  28|   nina@email.com| 55000|  2024-07-08|  Active|  Marketing|         6|   Tokyo|
|102|   Name| Age|unknown@email.com| 30000|  2024-07-08|  Status| Department|         1|Location|
|102|    Bob|  25|un

In [87]:
from pyspark.sql.functions import when, col

df_r = df_r.withColumn("Age", when(col("Age").isNull() | (col("Age") == ""), 25).otherwise(col("Age").cast("int"))) \
           .withColumn("Status", when(col("Status").isNull() | (col("Status") == ""), "Active").otherwise(col("Status").cast("string")))

df_r.show()

+---+-------+----+-----------------+------+------------+--------+-----------+----------+--------+
| ID|   Name| Age|            Email|Salary|Joining_Date|  Status| Department|Experience|LOCATION|
+---+-------+----+-----------------+------+------------+--------+-----------+----------+--------+
|101|  Alice|  25|  alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|New York|
|103|  Alice|  25|  alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|New York|
|108|  Fiona|  26|  fiona@email.com| 55000|  2023-11-05|    NULL|    Finance|         6|  Sydney|
|110|  Helen|NULL|  helen@email.com| 45000|  2018-06-07|Inactive|  Marketing|         8|   Dubai|
|111|   John|  25|   john@email.com| 80000|  2021-11-10|  Active|         IT|         3|New York|
|115|   Nina|  28|   nina@email.com| 55000|  2024-07-08|  Active|  Marketing|         6|   Tokyo|
|102|   Name|NULL|unknown@email.com| 30000|  2024-07-08|  Status| Department|         1|Location|
|102|    Bob|  25|un

In [98]:
df_r = df_r.na.fill({"Age": 25, "Status": "Active"})

In [99]:
df_r.show()

+---+-------+----+-----------------+------+------------+--------+-----------+----------+--------+
| ID|   Name| Age|            Email|Salary|Joining_Date|  Status| Department|Experience|LOCATION|
+---+-------+----+-----------------+------+------------+--------+-----------+----------+--------+
|101|  Alice|  25|  alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|New York|
|103|  Alice|  25|  alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|New York|
|108|  Fiona|  26|  fiona@email.com| 55000|  2023-11-05|    NULL|    Finance|         6|  Sydney|
|110|  Helen|NULL|  helen@email.com| 45000|  2018-06-07|Inactive|  Marketing|         8|   Dubai|
|111|   John|  25|   john@email.com| 80000|  2021-11-10|  Active|         IT|         3|New York|
|115|   Nina|  28|   nina@email.com| 55000|  2024-07-08|  Active|  Marketing|         6|   Tokyo|
|102|   Name| Age|unknown@email.com| 30000|  2024-07-08|  Status| Department|         1|Location|
|102|    Bob|  25|un

In [100]:
df_r = df_r.withColumn("Status", when(col("Status").isNull() | (trim(col("Status")) == " ") | (col("Status") == "None"), "Active").otherwise(col("Status")))

In [101]:
df_r.show()

+---+-------+----+-----------------+------+------------+--------+-----------+----------+--------+
| ID|   Name| Age|            Email|Salary|Joining_Date|  Status| Department|Experience|LOCATION|
+---+-------+----+-----------------+------+------------+--------+-----------+----------+--------+
|101|  Alice|  25|  alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|New York|
|103|  Alice|  25|  alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|New York|
|108|  Fiona|  26|  fiona@email.com| 55000|  2023-11-05|    NULL|    Finance|         6|  Sydney|
|110|  Helen|NULL|  helen@email.com| 45000|  2018-06-07|Inactive|  Marketing|         8|   Dubai|
|111|   John|  25|   john@email.com| 80000|  2021-11-10|  Active|         IT|         3|New York|
|115|   Nina|  28|   nina@email.com| 55000|  2024-07-08|  Active|  Marketing|         6|   Tokyo|
|102|   Name| Age|unknown@email.com| 30000|  2024-07-08|  Status| Department|         1|Location|
|102|    Bob|  25|un

In [102]:
df_final_data = df_r.select("*")

In [103]:
df_final_data.show()

+---+-------+----+-----------------+------+------------+--------+-----------+----------+--------+
| ID|   Name| Age|            Email|Salary|Joining_Date|  Status| Department|Experience|LOCATION|
+---+-------+----+-----------------+------+------------+--------+-----------+----------+--------+
|101|  Alice|  25|  alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|New York|
|103|  Alice|  25|  alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|New York|
|108|  Fiona|  26|  fiona@email.com| 55000|  2023-11-05|    NULL|    Finance|         6|  Sydney|
|110|  Helen|NULL|  helen@email.com| 45000|  2018-06-07|Inactive|  Marketing|         8|   Dubai|
|111|   John|  25|   john@email.com| 80000|  2021-11-10|  Active|         IT|         3|New York|
|115|   Nina|  28|   nina@email.com| 55000|  2024-07-08|  Active|  Marketing|         6|   Tokyo|
|102|   Name| Age|unknown@email.com| 30000|  2024-07-08|  Status| Department|         1|Location|
|102|    Bob|  25|un

In [104]:
df_final_data.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: string (nullable = false)
 |-- Email: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Joining_Date: string (nullable = true)
 |-- Status: string (nullable = false)
 |-- Department: string (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- LOCATION: string (nullable = true)



In [105]:
spark.sql("""

CREATE TABLE Stage_temp_table(
    ID INT,
    Name STRING,
    Age STRING,
    Email STRING,
    Salary INT,
    Joining_Date STRING,
    Status STRING,
    Department STRING,
    Experience INT,
    LOCATION STRING
)ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION 'hdfs://localhost:9000/datasets/'
""")

df_final_data.write.mode("overwrite").saveAsTable("Stage_temp_table")


In [106]:
stg = spark.sql("SELECT * FROM Stage_temp_table")

In [107]:
stg.show()

+---+-------+----+-----------------+------+------------+--------+-----------+----------+--------+
| ID|   Name| Age|            Email|Salary|Joining_Date|  Status| Department|Experience|LOCATION|
+---+-------+----+-----------------+------+------------+--------+-----------+----------+--------+
|101|  Alice|  25|  alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|New York|
|103|  Alice|  25|  alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|New York|
|108|  Fiona|  26|  fiona@email.com| 55000|  2023-11-05|    NULL|    Finance|         6|  Sydney|
|110|  Helen|NULL|  helen@email.com| 45000|  2018-06-07|Inactive|  Marketing|         8|   Dubai|
|111|   John|  25|   john@email.com| 80000|  2021-11-10|  Active|         IT|         3|New York|
|115|   Nina|  28|   nina@email.com| 55000|  2024-07-08|  Active|  Marketing|         6|   Tokyo|
|102|   Name| Age|unknown@email.com| 30000|  2024-07-08|  Status| Department|         1|Location|
|102|    Bob|  25|un

In [108]:
# DQ_checks

from pyspark.sql.functions import col

df_null_check = stg.filter(
    col("ID").isNull() | 
    col("Name").isNull() | 
    col("Age").isNull() | 
    col("Email").isNull() | 
    col("Salary").isNull() | 
    col("Joining_Date").isNull()
)
print("Rows with NULL values:", df_null_check.count())
df_null_check.show()

Rows with NULL values: 0
+---+----+---+-----+------+------------+------+----------+----------+--------+
| ID|Name|Age|Email|Salary|Joining_Date|Status|Department|Experience|LOCATION|
+---+----+---+-----+------+------------+------+----------+----------+--------+
+---+----+---+-----+------+------------+------+----------+----------+--------+



In [109]:
df_invalid_email = stg.filter(
    ~col("Email").rlike("^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\\.[a-zA-Z0-9-.]+$")
)

print("Invalid Email Records:", df_invalid_email.count())
df_invalid_email.show()

Invalid Email Records: 0
+---+----+---+-----+------+------------+------+----------+----------+--------+
| ID|Name|Age|Email|Salary|Joining_Date|Status|Department|Experience|LOCATION|
+---+----+---+-----+------+------------+------+----------+----------+--------+
+---+----+---+-----+------+------------+------+----------+----------+--------+



In [111]:
df_invalid_salary = stg.filter((col("Salary") <= 0))
df_invalid_age = df_final_data.filter((col("Age") < 18) | (col("Age") > 100))

print("Invalid Salary Records:", df_invalid_salary.count())
df_invalid_salary.show()

print("Invalid Age Records:", df_invalid_age.count())
df_invalid_age.show()

Invalid Salary Records: 0
+---+----+---+-----+------+------------+------+----------+----------+--------+
| ID|Name|Age|Email|Salary|Joining_Date|Status|Department|Experience|LOCATION|
+---+----+---+-----+------+------------+------+----------+----------+--------+
+---+----+---+-----+------+------------+------+----------+----------+--------+

Invalid Age Records: 0
+---+----+---+-----+------+------------+------+----------+----------+--------+
| ID|Name|Age|Email|Salary|Joining_Date|Status|Department|Experience|LOCATION|
+---+----+---+-----+------+------------+------+----------+----------+--------+
+---+----+---+-----+------+------------+------+----------+----------+--------+



In [112]:
from pyspark.sql.functions import to_date

df_invalid_date = stg.filter(
    to_date(col("Joining_Date"), "yyyy-MM-dd").isNull()
)

print("Invalid Date Records:", df_invalid_date.count())
df_invalid_date.show()

Invalid Date Records: 0
+---+----+---+-----+------+------------+------+----------+----------+--------+
| ID|Name|Age|Email|Salary|Joining_Date|Status|Department|Experience|LOCATION|
+---+----+---+-----+------+------------+------+----------+----------+--------+
+---+----+---+-----+------+------------+------+----------+----------+--------+



In [113]:
df_final_data = stg.dropDuplicates(["ID"])

In [114]:
df_final_data.show()

+---+-------+----+-----------------+------+------------+--------+-----------+----------+--------+
| ID|   Name| Age|            Email|Salary|Joining_Date|  Status| Department|Experience|LOCATION|
+---+-------+----+-----------------+------+------------+--------+-----------+----------+--------+
|101|  Alice|  25|  alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|New York|
|102|   Name| Age|unknown@email.com| 30000|  2024-07-08|  Status| Department|         1|Location|
|103|  Alice|  25|  alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|New York|
|104|Charlie|  28|charlie@email.com| 30000|  2021-05-10|Inactive|Engineering|         7|  Berlin|
|106|   NULL|  30|unknown@email.com| 70000|  2024-09-18|  Active|      Sales|         4|   Tokyo|
|108|  Fiona|  26|  fiona@email.com| 55000|  2023-11-05|    NULL|    Finance|         6|  Sydney|
|109|   Gary|  27|unknown@email.com| 75000|  2020-02-29|  Active|         HR|         2|  Mumbai|
|110|  Helen|NULL|  

In [116]:
df_invalid_status = stg.filter(~col("Status").isin(["Active", "Inactive"]))
print("Invalid Status Records:", df_invalid_status.count())
df_invalid_status.show()

Invalid Status Records: 2
+---+-----+---+-----------------+------+------------+------+----------+----------+--------+
| ID| Name|Age|            Email|Salary|Joining_Date|Status|Department|Experience|LOCATION|
+---+-----+---+-----------------+------+------------+------+----------+----------+--------+
|108|Fiona| 26|  fiona@email.com| 55000|  2023-11-05|  NULL|   Finance|         6|  Sydney|
|102| Name|Age|unknown@email.com| 30000|  2024-07-08|Status|Department|         1|Location|
+---+-----+---+-----------------+------+------------+------+----------+----------+--------+



In [118]:
from pyspark.sql.functions import when, col, trim

df_cleaned = stg.withColumn("Status", 
    when(col("Status").isNull() | (trim(col("Status")) == "") | (~col("Status").isin("Active", "Inactive")), "Active")
    .otherwise(col("Status"))
)

df_cleaned.show()

+---+-------+----+-----------------+------+------------+--------+-----------+----------+--------+
| ID|   Name| Age|            Email|Salary|Joining_Date|  Status| Department|Experience|LOCATION|
+---+-------+----+-----------------+------+------------+--------+-----------+----------+--------+
|101|  Alice|  25|  alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|New York|
|103|  Alice|  25|  alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|New York|
|108|  Fiona|  26|  fiona@email.com| 55000|  2023-11-05|  Active|    Finance|         6|  Sydney|
|110|  Helen|NULL|  helen@email.com| 45000|  2018-06-07|Inactive|  Marketing|         8|   Dubai|
|111|   John|  25|   john@email.com| 80000|  2021-11-10|  Active|         IT|         3|New York|
|115|   Nina|  28|   nina@email.com| 55000|  2024-07-08|  Active|  Marketing|         6|   Tokyo|
|102|   Name| Age|unknown@email.com| 30000|  2024-07-08|  Active| Department|         1|Location|
|102|    Bob|  25|un

In [126]:
df_invalid_status = df_cleaned.filter(~col("Status").isin(["Active", "Inactive"]))
print("Invalid Status Records:", df_invalid_status.count())
df_invalid_status.show()

Invalid Status Records: 0
+---+----+---+-----+------+------------+------+----------+----------+--------+
| ID|Name|Age|Email|Salary|Joining_Date|Status|Department|Experience|LOCATION|
+---+----+---+-----+------+------------+------+----------+----------+--------+
+---+----+---+-----+------+------------+------+----------+----------+--------+



In [127]:
df_cleaned.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Joining_Date: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- LOCATION: string (nullable = true)



In [128]:
spark.sql("""

CREATE TABLE Final_table(
    ID INT,
    Name STRING,
    Age STRING,
    Email STRING,
    Salary INT,
    Joining_Date STRING,
    Status STRING,
    Department STRING,
    Experience INT,
    LOCATION STRING
)ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION 'hdfs://localhost:9000/datasets/'
""")

df_cleaned.write.mode("overwrite").saveAsTable("Final_table")


In [129]:
Final_table_data = spark.sql("SELECT * FROM Final_table")

In [130]:
Final_table_data.show()

+---+-------+----+-----------------+------+------------+--------+-----------+----------+--------+
| ID|   Name| Age|            Email|Salary|Joining_Date|  Status| Department|Experience|LOCATION|
+---+-------+----+-----------------+------+------------+--------+-----------+----------+--------+
|101|  Alice|  25|  alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|New York|
|103|  Alice|  25|  alice@email.com| 50000|  2022-06-15|  Active|         HR|         3|New York|
|108|  Fiona|  26|  fiona@email.com| 55000|  2023-11-05|  Active|    Finance|         6|  Sydney|
|110|  Helen|NULL|  helen@email.com| 45000|  2018-06-07|Inactive|  Marketing|         8|   Dubai|
|111|   John|  25|   john@email.com| 80000|  2021-11-10|  Active|         IT|         3|New York|
|115|   Nina|  28|   nina@email.com| 55000|  2024-07-08|  Active|  Marketing|         6|   Tokyo|
|102|   Name| Age|unknown@email.com| 30000|  2024-07-08|  Active| Department|         1|Location|
|102|    Bob|  25|un

In [131]:
spark.stop()